In [ ]:
# Question Answering using SPA

# Now we will build this model again, using the spa (semantic pointer 
# architecture) package built into Nengo 2.0. You will see that using the spa 
# package considerably simplifies model construction and visualization through 
# nengo_gui.

# Press the play button to run the simulation.
# Graphs show the colour, shape and cue inputs. The last graph 
# shows that the output is most similar to the semantic pointer bound to the 
# current cue. For example, when RED and CIRCLE are being bound 
# and the cue is CIRCLE, the output is most similar to RED.

# Setup the environment
import nengo
import nengo_spa as spa
from nengo.spa import Vocabulary
import numpy as np

d = 32  # the dimensionality of the vectors
rng = np.random.RandomState(7)
vocab = spa.Vocabulary(dimensions=d, pointer_gen=rng, max_similarity=0.1)
vocab.populate("CIRCLE;BLUE;RED;SQUARE")
vocab.add('ZERO', [0]*d)

with spa.Network(vocabs=[vocab]) as model:
    
    D = spa.State(d, label="bound")
    E = spa.State(d, label="output")
  
    #function for providing color input
    def color_input(t):
        if (t // 0.5) % 2 == 0:
            return 'RED'
        else:
            return 'BLUE'
        
    #function for providing shape input
    def shape_input(t):
        if (t // 0.5) % 2 == 0:
            return 'CIRCLE'
        else:
            return 'SQUARE'

    #function for providing the cue
    def cue_input(t):
        sequence = ['ZERO', 'CIRCLE', 'RED', 'ZERO', 'SQUARE', 'BLUE']
        idx = int((t // (1. / len(sequence))) % len(sequence))
        return sequence[idx]
        
    A = spa.Transcode(color_input, label="color", output_vocab=d)
    B = spa.Transcode(shape_input, label="shape", output_vocab=d)
    C = spa.Transcode(cue_input, label="cue", output_vocab=d)
    
    A * B >> D
    D * ~C >> E